# Utils

> Various useful pieces

In [ ]:
#| default_exp settings

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import os
from pydantic_settings import BaseSettings
from pydantic import Field

from pathlib import Path

# Load environment variables from .env file
from dotenv import load_dotenv; load_dotenv();

## Set up logging

In [ ]:
#| export

import logging
import time
import colorlog

In [ ]:
#| exporti

# Store the last log time globally
_last_log_time = time.time()

class TimeDeltaLogFormatter(colorlog.ColoredFormatter):
    """Custom formatter that shows time delta since last log message instead of timestamp."""

    def format(self, record: logging.LogRecord) -> str:
        global _last_log_time
        current_time = time.time()
        delta = current_time - _last_log_time
        _last_log_time = current_time

        # Add the delta as a field to the record
        record.delta = f"{delta:.3f}"

        # Convert the pathname to be relative to lovely-docs.
        if hasattr(record, "pathname"):
            if "ipykernel" in record.pathname:
                record.pathname = "<ipykernel>"
            if "lovely-docs/" in record.pathname:
                record.pathname = record.pathname.split("lovely-docs/")[1]

        return super().format(record)


def setup_logging() -> None:
    """Set up logging for the application."""

    # Known flooders
    # logging.getLogger("httpx").setLevel(logging.WARNING)
    # logging.getLogger("anthropic").setLevel(logging.WARNING)
    # logging.getLogger("httpcore").setLevel(logging.WARNING)

    # Reset handlers to avoid duplicate logs
    logger = logging.getLogger()
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)


    console_handler = logging.StreamHandler()

    # Define color scheme
    log_colors = {
        'DEBUG': 'cyan',
        'INFO': 'green',
        'WARNING': 'yellow',
        'ERROR': 'red',
        'CRITICAL': 'red,bg_white',
    }

    # Create formatter with time delta and colors
    formatter = TimeDeltaLogFormatter(
        "+%(delta)ss %(log_color)s%(levelname)s%(reset)s %(blue)s%(pathname)s:"
        "%(lineno)d%(reset)s %(funcName)s %(message)s",
        log_colors=log_colors
    )

    console_handler.setFormatter(formatter)

    # Add handler to logger
    logger.addHandler(console_handler)

setup_logging()

## Settings

In [ ]:
#| export



class Source(BaseSettings):
    name: str
    comment: str|None = None
    ecosystems: list[str] = Field(default_factory=list)
    include: list[str] | None = None
    exclude: list[str] | None = None


class GitSource(Source):
    doc_dir: Path
    repo: str
    commit: str

class LLMTxtSource(Source):
    url: str
    # llms.txt will often come as markdown with a lot of nested sections.
    # We will split individual sub-sections as documentation pages / directories, up to depth.
    depth: int

class WebSource(Source):
    pass


class Settings(BaseSettings):
    model: str = "claude-haiku-4.5"
    git_dir: Path = Path("../git_dir") # Cloned git repos will be here
    output_dir: Path = Path("../../doc_db/")
    templates_dir: Path = Path("../templates")
    web_cache_dir: Path = Path("../web_cache/")
    project_root: Path = Path(__file__).parent.parent if "__file__" in globals() else Path.cwd()
    api_key: str = os.getenv("ANTHROPIC_API_KEY", "")

settings = Settings()